# CPS_SQL_01_Fundamentals


## Initial Setup (run once)

Connect to DuckDB and load CSV tables.

In [ ]:
%load_ext sql
%sql duckdb:///../practice.duckdb

In [ ]:
%%sql
CREATE OR REPLACE TABLE employees AS SELECT * FROM read_csv_auto('../datasets/employee_data_large.csv');
CREATE OR REPLACE TABLE sales AS SELECT * FROM read_csv_auto('../datasets/sales_large.csv');
CREATE OR REPLACE TABLE students AS SELECT * FROM read_csv_auto('../datasets/student_data_large.csv');
CREATE OR REPLACE TABLE departments AS SELECT * FROM read_csv_auto('../datasets/departments.csv');
CREATE OR REPLACE TABLE customers AS SELECT * FROM read_csv_auto('../datasets/customers.csv');
CREATE OR REPLACE TABLE orders AS SELECT * FROM read_csv_auto('../datasets/orders.csv');

## Setup (run once after kernel restart)
Reconnect to DuckDB.

In [ ]:
%reload_ext sql
%sql duckdb:///../practice.duckdb

In [ ]:
%config SqlMagic.displaylimit = None

## Section 1: Basic SELECT
Select all columns from `employees`.

In [ ]:
%%sql
SELECT * FROM employees

Display only the name, department, and salary columns

In [ ]:
%%sql
SELECt name, department, salary 
FROM employees

Display all employees sorted alphabetically by name

In [ ]:
%%sql
SELECT * 
FROM employees
ORDER BY name

Display all employees sorted by salary from highest to lowest

In [ ]:
%%sql 
SELECT *
FROM employees
ORDER BY salary DESC

Display only employees from the IT department

In [ ]:
%%sql
SELECT *
FROM employees
WHERE department = 'IT'

## Section 2: Handling NULLS
Find all employees with a missing salary

In [ ]:
%%sql
SELECT * 
FROM employees
WHERE salary IS NULL

Find all employees with a missing age

In [ ]:
%%sql
SELECT *
FROM employees
WHERE age IS NULL

Count how many employees are missing salary or age values

In [ ]:
%%sql
SELECT COUNT(*) AS missing_info
FROM employees
WHERE salary IS NULL OR age IS NULL

Show employees who have both age and salary present

In [ ]:
%%sql
SELECT *
FROM employees
WHERE salary IS NOT NULL AND age IS NOT NULL

Replace missing salaries with 0 in the output (do not update the table)

In [ ]:
%%sql
SELECT 
    employee_id,
    name,
    department,
    status,
    COALESCE(salary, 0) AS salary_filled
FROM employees


## Section 3: Aggregation
Count total number of employees

In [ ]:
%%sql
SELECT COUNT(employee_id) AS total_employees
FROM employees


Find average salary of employees (round to two decimal places)

In [ ]:
%%sql
SELECT ROUND(AVG(salary), 2) AS avg_salary
FROM employees
WHERE salary IS NOT NULL

Find the minimum and maximum salary

In [ ]:
%%sql
SELECT 
    MIN(salary) AS min_salary,
    MAX(salary) AS max_salary
FROM employees
WHERE salary IS NOT NULL

Count how many employees are in the Sales department

In [ ]:
%%sql
SELECT COUNT(employee_id) AS sales_count
FROM employees
WHERE department='Sales'


Find how much money the company is spending on salaries

In [ ]:
%%sql
SELECT ROUND(SUM(salary), 2) AS sum_of_salaries
FROM employees
WHERE salary IS NOT NULL

## Section 4: Filtering with WHERE, GROUP BY, HAVING
Find average salary of each department

In [ ]:
%%sql
SELECT
    department,
    ROUND(AVG(salary), 2) AS avg_salary
FROM employees
WHERE salary IS NOT NULL
GROUP BY department
    

Find employees who are not in the Sales or HR departments

In [ ]:
%%sql
SELECT *
FROM employees
WHERE department NOT IN ('Sales', 'HR')

Count employees by status

In [ ]:
%%sql
SELECT 
    status,
    COUNT(employee_id) AS employee_count
FROM employees
GROUP BY status

Show departments with at least 40 employees

In [ ]:
%%sql
SELECT 
    department,
    COUNT(employee_id) AS number_of_employees
FROM employees
GROUP BY department
HAVING COUNT(employee_id) >= 40

Find departments where the average salary is greater than 55,000

In [ ]:
%%sql
SELECT 
    department,
    ROUND(AVG(salary), 2) AS avg_salary
FROM employees
WHERE salary IS NOT NULL
GROUP BY department
HAVING avg_salary > 55000

## Section 5: Sorting & Ranking Basics
Show the top 10 highest-paid employees

In [ ]:
%%sql
SELECT *
FROM employees
WHERE SALARY IS NOT NULL
ORDER BY salary DESC
LIMIT 10


Show the bottom 10 lowest-paid employees (ignore NULL salaries)

In [ ]:
%%sql
SELECT *
FROM employees
WHERE salary IS NOT NULL
ORDER BY salary
LIMIT 10

Show the 5 youngest employees

In [ ]:
%%sql
SELECT *
FROM employees
WHERE age IS NOT NULL
ORDER BY age
LIMIT 5

Show the 5 oldest employees

In [ ]:
%%sql
SELECT *
FROM employees
WHERE age IS NOT NULL
ORDER BY age DESC
LIMIT 5


Show employees ordered by department, then by salary descending (treat NULL as 0)

In [ ]:
%%sql
SELECT 
    employee_id,
    name,
    department,
    status,
    COALESCE(salary, 0) AS salary_filled
FROM employees
ORDER BY department ASC, salary_filled DESC

## Section 6: Simple Derived Columns
Create a column called salary_in_thousands

In [ ]:
%%sql
SELECT
    *,
    ROUND(salary / 1000.0, 2) AS salary_in_thousands
FROM employees
WHERE salary IS NOT NULL

Create a column that labels employees as:

- "Missing" if salary is NULL

- "High Earner" if salary ≥ 100,000

- "Mid Earner" if salary between 60,000–99,999

- "Low Earner" otherwise

In [ ]:
%%sql
SELECT
    *,
    CASE
        WHEN salary IS NULL THEN 'Missing'
        WHEN salary >= 100000 THEN 'High Earner'
        WHEN salary >= 60000 THEN 'Mid Earner'
        ELSE 'Low Earner'
    END AS earner_type
FROM employees

Create a column that flags employees as "Missing Salary" or "Has Salary"

In [ ]:
%%sql
SELECT 
    *,
    CASE
        WHEN salary IS NULL THEN 'Missing Salary'
        ELSE 'Has Salary'
    END AS salary_flag
FROM employees

Round salaries to the nearest whole number in the output.

In [ ]:
%%sql
SELECT
    *,
    ROUND(salary, 0) AS salary_rounded
FROM employees
WHERE salary IS NOT NULL


Show each employees's salary minus the company's average salary

In [ ]:
%%sql
SELECT
    *,
    ROUND(salary - (SELECT AVG(salary) FROM employees WHERE salary IS NOT NULL), 2) AS salary_minus_avg
FROM employees
WHERE salary IS NOT NULL

## Section 7: Data Quality Checks
Count how many distinct departments exist

In [ ]:
%%sql
SELECT COUNT(DISTINCT department) AS distinct_departments
FROM employees
WHERE department IS NOT NULL

Count how many distinct statuses exist

In [ ]:
%%sql
SELECT COUNT(DISTINCT status) AS distinct_status
FROM employees
WHERE status IS NOT NULL

Find duplicate employee names (if any)

In [ ]:
%%sql
SELECT
    name,
    COUNT(*) AS name_count
FROM employees
GROUP BY name
HAVING COUNT(*) > 1
ORDER BY name_count DESC

Identify departments where any salary is NULL

In [ ]:
%%sql
SELECT
    department,
    COUNT(*) AS null_salary_count
FROM employees
WHERE salary IS NULL
GROUP BY department
ORDER BY null_salary_count DESC

Produce a small “data quality report” showing:

- total rows

- rows with missing salary

- rows with missing age

In [ ]:
%%sql
SELECT
    COUNT(*) AS total_rows,
    SUM(CASE WHEN salary IS NULL THEN 1 ELSE 0 END) AS rows_with_missing_salary,
    SUM(CASE WHEN age IS NULL THEN 1 ELSE 0 END) AS rows_with_missing_age
FROM employees